# 03 overlay

**in**: video, tracking csv, eval_ts, pass_df  
**out**: `data/render/final.mp4`

overlay stack (bottom to top):
1. source frame
2. pitch control voronoi (alpha=0.3)
3. xT zones (alpha=0.2)
4. pass arrows colored by risk
5. eval bar gauge (corner)

In [ ]:
# cell 1: load all data

from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import supervision as sv
from scipy.spatial import Voronoi
from ultralytics import YOLO

from sports.common.view import ViewTransformer
from sports.configs.soccer import SoccerPitchConfiguration

# paths
VIDEO_DIR = Path('../data/video')
TRACK_DIR = Path('../data/track')
OUT_DIR = Path('../data/out')
RENDER_DIR = Path('../data/render')
MODEL_DIR = Path('../data/models')
RENDER_DIR.mkdir(parents=True, exist_ok=True)

# find files
video_files = list(VIDEO_DIR.glob('*.mp4'))
if not video_files:
    raise FileNotFoundError(f'no video found in {VIDEO_DIR}')
VIDEO_PATH = video_files[0]
MATCH_NAME = VIDEO_PATH.stem

TRACK_CSV = TRACK_DIR / f'{MATCH_NAME}_track.csv'
EVAL_CSV = OUT_DIR / f'{MATCH_NAME}_eval_ts.csv'
PASS_CSV = OUT_DIR / f'{MATCH_NAME}_pass_df.csv'

print(f'video: {VIDEO_PATH}')
print(f'track: {TRACK_CSV}')
print(f'eval: {EVAL_CSV}')

# load data
track_df = pd.read_csv(TRACK_CSV)
eval_df = pd.read_csv(EVAL_CSV)
pass_df = pd.read_csv(PASS_CSV) if PASS_CSV.exists() else pd.DataFrame()

print(f'track: {len(track_df)} rows')
print(f'eval: {len(eval_df)} rows')
print(f'passes: {len(pass_df)} rows')

# video info
video_info = sv.VideoInfo.from_video_path(str(VIDEO_PATH))
FPS = video_info.fps
WIDTH = video_info.width
HEIGHT = video_info.height
print(f'video: {WIDTH}x{HEIGHT} @ {FPS} fps')

# pitch config (12000x7000 cm = 120x70 m)
PITCH_CFG = SoccerPitchConfiguration()
PITCH_X = 120.0  # meters
PITCH_Y = 70.0   # meters

In [ ]:
# cell 2: draw functions

# colors
TEAM_COLORS = {
    0: (33, 150, 243),   # blue (BGR)
    1: (244, 67, 54),    # red
    2: (255, 235, 59),   # yellow (goalkeeper)
    3: (0, 0, 0),        # black (referee)
    4: (255, 255, 255),  # white (ball)
}


def draw_pitch_control(frame, frame_df, vtf_inv, alpha=0.3):
    """
    Draw pitch control voronoi overlay.
    vtf_inv: inverse view transformer (pitch -> pixel)
    """
    overlay = np.zeros_like(frame, dtype=np.uint8)
    
    players = frame_df[(frame_df['team'].isin([0, 1])) & (frame_df['cls'] == 'player')]
    if len(players) < 4:
        return frame
    
    points = players[['x', 'y']].values
    teams = players['team'].values
    
    # add mirror points for bounded voronoi
    mirror = []
    for p in points:
        mirror.append([-p[0], p[1]])
        mirror.append([2*PITCH_X - p[0], p[1]])
        mirror.append([p[0], -p[1]])
        mirror.append([p[0], 2*PITCH_Y - p[1]])
    
    all_pts = np.vstack([points, mirror])
    
    try:
        vor = Voronoi(all_pts)
    except Exception:
        return frame
    
    # draw regions for original points
    for i in range(len(points)):
        region_idx = vor.point_region[i]
        if region_idx == -1:
            continue
        region = vor.regions[region_idx]
        if -1 in region or len(region) < 3:
            continue
        
        # clip polygon to pitch
        poly_pitch = vor.vertices[region]
        poly_pitch = np.clip(poly_pitch, [0, 0], [PITCH_X, PITCH_Y])
        
        # transform to pixel coords
        if vtf_inv is not None:
            poly_pixel = vtf_inv.transform_points(poly_pitch.astype(np.float32))
            poly_pixel = poly_pixel.astype(np.int32)
            
            color = TEAM_COLORS.get(int(teams[i]), (128, 128, 128))
            cv2.fillPoly(overlay, [poly_pixel], color)
    
    # blend
    return cv2.addWeighted(frame, 1.0, overlay, alpha, 0)


def draw_xt_heat(frame, ball_x, ball_y, vtf_inv, alpha=0.2):
    """
    Draw xT heatmap around ball position.
    """
    if ball_x is None or vtf_inv is None:
        return frame
    
    overlay = np.zeros_like(frame, dtype=np.uint8)
    
    # create gradient around ball
    grid_size = 10
    for gx in range(0, int(PITCH_X), grid_size):
        for gy in range(0, int(PITCH_Y), grid_size):
            cx, cy = gx + grid_size/2, gy + grid_size/2
            xt = (cx / PITCH_X) ** 1.8 * np.exp(-((cy - PITCH_Y/2)**2) / (2*18**2))
            
            # color based on xT (green to red)
            r = int(255 * xt)
            g = int(255 * (1 - xt))
            color = (0, g, r)  # BGR
            
            # transform corners to pixel
            corners = np.array([
                [gx, gy], [gx + grid_size, gy],
                [gx + grid_size, gy + grid_size], [gx, gy + grid_size]
            ], dtype=np.float32)
            
            try:
                pixel_corners = vtf_inv.transform_points(corners).astype(np.int32)
                cv2.fillPoly(overlay, [pixel_corners], color)
            except Exception:
                pass
    
    return cv2.addWeighted(frame, 1.0, overlay, alpha, 0)


def draw_pass_arrows(frame, pass_df, current_frame, vtf_inv, window=30):
    """
    Draw recent pass arrows colored by risk.
    """
    if len(pass_df) == 0 or vtf_inv is None:
        return frame
    
    recent = pass_df[
        (pass_df['frame_start'] <= current_frame) & 
        (pass_df['frame_end'] >= current_frame - window)
    ]
    
    for _, p in recent.iterrows():
        start = np.array([[p['sx'], p['sy']]], dtype=np.float32)
        end = np.array([[p['ex'], p['ey']]], dtype=np.float32)
        
        try:
            start_px = vtf_inv.transform_points(start)[0].astype(int)
            end_px = vtf_inv.transform_points(end)[0].astype(int)
        except Exception:
            continue
        
        # color by success probability
        p_pass = p.get('p_pass', 0.5)
        if p_pass >= 0.72:
            color = (0, 255, 0)  # green - safe
        elif p_pass >= 0.45:
            color = (0, 255, 255)  # yellow - medium
        else:
            color = (0, 0, 255)  # red - risky
        
        cv2.arrowedLine(frame, tuple(start_px), tuple(end_px), color, 3, tipLength=0.2)
    
    return frame


def draw_eval_gauge(frame, eval_val, x=50, y=50, w=200, h=30):
    """
    Draw eval bar gauge in corner.
    eval_val: -100 to 100
    """
    # background
    cv2.rectangle(frame, (x, y), (x + w, y + h), (50, 50, 50), -1)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    
    # center line
    cx = x + w // 2
    cv2.line(frame, (cx, y), (cx, y + h), (200, 200, 200), 2)
    
    # fill based on eval
    fill_w = int((abs(eval_val) / 100.0) * (w // 2))
    
    if eval_val >= 0:
        color = TEAM_COLORS[0]  # blue
        cv2.rectangle(frame, (cx, y + 2), (cx + fill_w, y + h - 2), color, -1)
    else:
        color = TEAM_COLORS[1]  # red
        cv2.rectangle(frame, (cx - fill_w, y + 2), (cx, y + h - 2), color, -1)
    
    # text
    text = f'{int(eval_val):+d}'
    cv2.putText(frame, text, (x + w + 10, y + h - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, 'EVAL', (x, y - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    
    return frame


print('draw functions defined')

In [ ]:
# cell 3: frame-by-frame overlay render

# load pitch keypoint model for view transform
KP_MODEL_PATH = MODEL_DIR / 'football-pitch-detection.pt'
kp_model = YOLO(str(KP_MODEL_PATH))

# output video
OUTPUT_PATH = RENDER_DIR / f'{MATCH_NAME}_overlay.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(str(OUTPUT_PATH), fourcc, FPS, (WIDTH, HEIGHT))

# create eval lookup
eval_lookup = dict(zip(eval_df['frame'], eval_df['eval_bar']))

# convert pitch config vertices from cm to meters for matching our tracking data
pitch_vertices_m = np.array(PITCH_CFG.vertices) / 100.0  # cm -> m

# render
frame_idx = 0
rendered = 0

for frame in sv.get_video_frames_generator(str(VIDEO_PATH)):
    # get eval for this frame
    eval_val = eval_lookup.get(frame_idx, 0)
    
    # get tracking data for this frame
    frame_df = track_df[track_df['frame'] == frame_idx]
    
    # get view transform
    vtf_inv = None
    try:
        kp_result = kp_model(frame, verbose=False)[0]
        kps = sv.KeyPoints.from_ultralytics(kp_result)
        if len(kps.xy) > 0:
            mask = (kps.xy[0][:, 0] > 1) & (kps.xy[0][:, 1] > 1)
            if mask.sum() >= 4:
                # inverse transform: pitch (meters) -> pixel
                vtf_inv = ViewTransformer(
                    source=pitch_vertices_m[mask].astype(np.float32),
                    target=kps.xy[0][mask].astype(np.float32),
                )
    except Exception:
        pass
    
    # apply overlays
    if len(frame_df) > 0 and vtf_inv is not None:
        # pitch control (alpha=0.3)
        frame = draw_pitch_control(frame, frame_df, vtf_inv, alpha=0.25)
        
        # xT heat (alpha=0.2) - optional, can be slow
        # ball = frame_df[frame_df['cls'] == 'ball']
        # if len(ball) > 0:
        #     frame = draw_xt_heat(frame, ball['x'].iloc[0], ball['y'].iloc[0], vtf_inv, alpha=0.15)
        
        # pass arrows
        frame = draw_pass_arrows(frame, pass_df, frame_idx, vtf_inv)
    
    # eval gauge (always)
    frame = draw_eval_gauge(frame, eval_val, x=50, y=50)
    
    writer.write(frame)
    rendered += 1
    frame_idx += 1
    
    if frame_idx % 250 == 0:
        print(f'rendered {frame_idx} frames')

writer.release()
print(f'\noverlay saved: {OUTPUT_PATH}')
print(f'rendered {rendered} frames')

In [ ]:
# cell 4: ffmpeg compose with source video

import subprocess

FINAL_PATH = RENDER_DIR / f'{MATCH_NAME}_final.mp4'

# if you want semi-transparent overlay on source:
# ffmpeg -y -i src.mp4 -i overlay.mp4 \
#   -filter_complex "[1:v]format=rgba,colorchannelmixer=aa=0.5[ov];[0:v][ov]overlay" \
#   -c:v libx264 -crf 20 final.mp4

# for now, just copy the overlay (which already has source baked in)
cmd = [
    'ffmpeg', '-y',
    '-i', str(OUTPUT_PATH),
    '-c:v', 'libx264',
    '-preset', 'fast',
    '-crf', '22',
    '-pix_fmt', 'yuv420p',
    str(FINAL_PATH)
]

print(f'running: {" ".join(cmd)}')
result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode == 0:
    print(f'final video: {FINAL_PATH}')
else:
    print(f'ffmpeg error: {result.stderr}')

In [ ]:
# cell 5: export clips

def extract_clip(input_path, output_path, start_sec, duration_sec):
    """Extract a clip from video."""
    cmd = [
        'ffmpeg', '-y',
        '-ss', str(start_sec),
        '-i', str(input_path),
        '-t', str(duration_sec),
        '-c:v', 'libx264',
        '-preset', 'fast',
        '-crf', '22',
        str(output_path)
    ]
    subprocess.run(cmd, capture_output=True)
    print(f'clip saved: {output_path}')


# find interesting moments (big eval swings)
eval_df['eval_change'] = eval_df['eval_bar'].diff().abs()
big_swings = eval_df.nlargest(5, 'eval_change')

print('biggest eval swings:')
print(big_swings[['frame', 't_sec', 'eval_bar', 'eval_change']])

# export clips around swings
CLIPS_DIR = RENDER_DIR / 'clips'
CLIPS_DIR.mkdir(exist_ok=True)

for i, (_, row) in enumerate(big_swings.iterrows()):
    start = max(0, row['t_sec'] - 10)
    clip_path = CLIPS_DIR / f'{MATCH_NAME}_highlight_{i+1}.mp4'
    extract_clip(FINAL_PATH, clip_path, start, 20)

print(f'\n{len(big_swings)} clips exported to {CLIPS_DIR}')

## verification checklist

- [ ] overlay renders without crashing
- [ ] pitch control visible but not overwhelming
- [ ] eval bar gauge updates smoothly
- [ ] final.mp4 plays correctly